<div style="direction:rtl;line-height:300%;">
	<font face="XB Zar" size=5>
		<div align=center>
			<font face="IranNastaliq" size=30>
				<p></p>
				<p></p>
به نام خدا
				<p></p>
			</font>
			<font color=#FF7500>
دانشگاه صنعتی شریف - دانشکده مهندسی کامپیوتر
            </font>
			<p></p>
			<font color=blue>
هوش مصنوعی تمرین عملی پنجم
            </font>
			<br />
		<br>
            مهدی سلمانی صالح آبادی
            <br>
            98105824
        </div>
	</font>
</div>

<div dir='rtl'>
    <h1>import کردن کتابخانه‌های لازم</h1>
</div>

In [1]:
!pip install gym

In [2]:
from random import randint, random, uniform
import gym
import time

In [3]:
env = gym.make('MountainCar-v0')

<div dir='rtl'>
    <h1>بررسی مسئله</h1>
    <br>
    در هرحله برای حالت ماشین از دو منظر تعریف می‌شود.
    <ul>
        <li>
            سرعت:
            عددی بین 
            $-0.07$ تا
            $0.07$
            است.
            چون این مقدار پیوسته است پس تعداد ناشمارا حالت مختلف دارد که برای ما کاربردی نیست.
            برای همین بازه مذکور را به زیر‌بازه‌هایی افراز می‌کنیم
            و حالتهایی که سرعتشان در یک بازه قرار گیرند را دارای سرعت یکسان فرض می‌گیریم.  
            برای این کار نیز اعداد را تا دو رقم اعشار گرد می‌کنیم و آنهایی که رقم
             دوم اعشارشان با هم برابر است را در یک بازه قرار می‌دهیم.
            دقت کنید که به دلیل دقت بازه مذکور آنها را تا یک رقم اعشار گرد نکردیم و همچنین به علت جلوگیری از زیاد شدن حالات آن را فقط تا دو رقم گرد کردیم و دقتمان را بیشتر نکردیم.
        </li>
        <li>
            مکان:
            عددی بین 
            $-1.2$ تا
            $0.6$
            است.
            چون این مقدار پیوسته است پس تعداد ناشمارا حالت مختلف دارد که برای ما کاربردی نیست.
            برای همین بازه مذکور را به زیر‌بازه‌هایی افراز می‌کنیم
            و حالتهایی که مکانشان در یک بازه قرار گیرند را دارای سرعت یکسان فرض می‌گیریم. 
            برای این کار نیز اعداد را تا یک رقم اعشار گرد می‌کینم.
        </li>
    </ul>
</div>

In [4]:
def get_position_state(position):
    return round(position * 10) - round(env.observation_space.low[0] * 10)
def get_velocity_state(velocity):
    return round(velocity * 100) - round(env.observation_space.low[1] * 100)
print(f'low_position_state: {get_position_state(-1.2)} high_position_state: {get_position_state(0.6)}')
print(f'low_velocity_state: {get_velocity_state(-0.07)} high_velocity_state: {get_velocity_state(0.07)}')

low_position_state: 0 high_position_state: 18
low_velocity_state: 0 high_velocity_state: 14


<div dir='rtl'>
    حال دقت کینم که تعداد هر حالات یه زوج مرتب از مکان و سرعت می‌باشد.
    پس تعداد کل حالات به شرح زیر است:
</div>

In [5]:
num_velocity_state = get_velocity_state(env.observation_space.high[1]) + 1 
num_position_state = get_position_state(env.observation_space.high[0]) + 1 
print('total_state: ', num_velocity_state * num_position_state)

total_state:  285


<div dir='rtl'>
    حال دقت کنید در این مسئله می‌خواهیم از 
    $Q Learning$
    استفاده کینم.
    پس آرایه 
    Q_values
    که در آن مقادیر
    $Q(S, a)$
    نگه‌داری می‌شود را تعریف می‌کینم.
    دقت کنید که چون 3 اکشن در هر حالت داریم پس 
    Q_values
    یک آرایه
    num_position_state
    در
    num_velocity_state
    در
    3
    می‌شود.
</div>

In [6]:
Q_values = [[[uniform(-1, 1) for k in range(3)]for j in range(num_velocity_state)] for i in range(num_position_state)]

<div dir='rtl'>
    <h1>نوشتن تابع یادگیری</h1>
    در زیر تابعی که یک اپیزود را یاد بگیرد مشاهده می‌کنید.
    در این‌جا از روش
    epsilon greedy
    استفاده می‌کینم.
    یعنی در هر اپیزود با احتمال
    epsilon
    حرکتی تصادفی می‌کنیم.
    (exploration)
    و با احتمال
    1 - epsilon
    مطابق سیاستمان بهترین حرکتمان را انجام می‌دهیم.
    (exploitation)
    پس از انجام حرکت به حالتی وارد می‌شویم که کتابخانه 
    gym
    آن حالت و اینکه آیا آن حالت ترمینال است یا نه را به ما می‌دهد. 
    (یعنی
    isTerminal
    وقتی زمان اپیزود تمام شود و یا به موقعیت هدف برسیم
    true
    می‌شود.)
    درنهایت نیز مطابق نکات گفته شده در کلاس
    Q_values
    را آپدیت می‌کینم.
    <br>
    برای راحتی کار نیز، یک ورودی برای این تابع تعریف شده‌است که نشان می‌دهد که باید یادگیری انجام دهد یا نه.
    اگر 
    learning == True
    آنگاه یادگیری انجام می‌شود و در غیر این حالت صرفا یک اپیزود با 
    Q_values
    فعلی اجرا می‌شود و نتیجه نهایی آن که 
    total_reward
    باشد خارج می‌شود.
</div>

In [7]:
def episode_learning(Q_values, discount, epsilon, lr, learning=True):
    initial_state = env.reset()
    initial_state = [get_position_state(initial_state[0]), get_velocity_state(initial_state[1])]
    current_state = initial_state
    
    isTerminal = False
    total_reward = 0
        
    while isTerminal != True: 
#         if not learning:
#             env.render()
        p = random()
        if p < epsilon and learning:
            action = randint(0, 2)
        else:
            values = Q_values[current_state[0]][current_state[1]]
            action = values.index(max(values))
        
        [position, velocity], reward, isTerminal, _ = env.step(action)
        next_state = [get_position_state(position), get_velocity_state(velocity)]
        total_reward += reward
        if learning:
            if isTerminal and position >= 0.5:
                Q_values[current_state[0]][current_state[1]][action] = reward

            else:
                val = Q_values[next_state[0]][next_state[1]]
                sample = reward + discount * max(val)
                Q_values[current_state[0]][current_state[1]][action] = (1 - lr) * Q_values[current_state[0]][current_state[1]][action] + lr * sample

        current_state = next_state
    if learning:
        return Q_values
    else:
        return total_reward

<div dir='rtl'>
    در نهایت در کد زیر تابعی که کل فرایند
    Q_learning
    را در 15 دقیقه انجام می‌دهد را می‌بینید.
    این تابع زمان شروع قبل از 
    learning
    را می‌گیرد و سپس تا 889 ثانیه یادگیری را انجام می‌دهد.
    (آن یک ثانیه کمتر جهت اطمینان است.)
    نکته مهم بحث مربوط به 
    epsilon greedy
    است.
    می‌دانیم در این روش باید به مرور زمان میزان
    exploration
    کمتر شود و برای همین باید میزان
    epsilon
    باید در هر مرحله کمتر شود که به سیاست کاهش 
    epsilon, epsilon decay
    می‌گوییم.
    برای این مسئله هر بار اپسیلون را در یک عدد کوچکتر از یک ضرب می‌کنیم.
</div>

In [8]:
def Q_learning(Q_values, discount, epsilon, lr, epsilon_lr, start_time):
    while time.time() - start_time < 15 * 60 - 1:
        Q_values = episode_learning(Q_values, discount, epsilon, lr)
        epsilon = epsilon * epsilon_lr
    return Q_values

<div dir='rtl'>
    در کد زیر نیز با توجه به تابع بالا 15 دقیقه یادگیری انجام می‌شود.
    (زمان یادگیری بر حسب ثانیه)
</div>

In [9]:
starting_time = time.time()
Q_values = Q_learning(Q_values, 0.9, 0.8, 0.2, 0.9993, starting_time)
print('total learning time: ', time.time() - starting_time)

total learning time:  899.0007982254028


<div dir='rtl'>
    در کد زیر reward
     در یک مرحله را می‌بینید.
</div>

In [12]:
reward = episode_learning(Q_values, 0.9, 0.8, 0.2, learning=False)
print('test reward for one episode: ', reward)

test reward for one episode:  -135.0


<div dir='rtl'>
    در کد زیر نیز میانگین برای 
    100
    اجرا پس از یادگیری را می‌بینید.
</div>

In [13]:
reward = 0
for i in range(100):
    reward += episode_learning(Q_values, 0.9, 0.8, 0.2, learning=False)
print(reward / 100)

-144.12


<div dir='rtl'>
   در زیر مقدار 
   Q_values
    را می‌بینید.
</div>

In [14]:
Q_values

[[[-0.7891205878317054, -0.31781065063044767, 0.413606058111474],
  [-9.908962807745779, -9.886746114706773, -9.909566026976783],
  [-9.88715142679353, -9.948727581376085, -9.948716172564154],
  [-9.949774135922821, -9.890775824732517, -9.950069535818304],
  [-9.949786185996995, -9.9499342456585, -9.884653974680926],
  [-9.901636353896809, -9.955167425136294, -9.955013543878948],
  [-9.955016949859061, -9.900961477269096, -9.95506565346002],
  [-9.94659062938113, -9.946316406417552, -9.872922718138698],
  [-9.939761474347808, -9.939550859459557, -9.846283668311008],
  [-9.917293999880133, -9.913013522968203, -9.801142436126124],
  [0.054179145488992386, 0.4502908582835654, 0.03002416307713185],
  [0.4114015348698443, 0.18994448805104658, 0.7710714560061163],
  [0.08559091959550846, -0.09773135351724682, -0.907891186525787],
  [-0.5534508175785189, -0.15942593351300482, 0.5321784700442942],
  [0.4762329653514694, 0.027012114046678803, -0.5455646299829027]],
 [[0.3341044397528645, -0.118

<div dir='rtl'>
   در زیر نیز 
    policy
    انتخابی را در هر حالت مشاهده می‌کنید.
</div>

In [16]:
policy = [[Q_values[i][j].index(max(Q_values[i][j])) for j in range(num_velocity_state)] for i in range(num_position_state)]
policy

[[2, 1, 0, 1, 2, 0, 1, 2, 2, 2, 1, 2, 0, 2, 0],
 [2, 1, 0, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2],
 [1, 0, 1, 0, 2, 1, 1, 2, 2, 2, 2, 0, 0, 2, 2],
 [2, 0, 0, 2, 1, 2, 1, 1, 2, 0, 2, 2, 2, 0, 1],
 [0, 0, 0, 2, 0, 0, 1, 1, 1, 2, 2, 2, 2, 0, 1],
 [2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2],
 [2, 0, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0],
 [2, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2],
 [1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 2, 1, 1, 0],
 [1, 1, 0, 0, 2, 0, 0, 0, 0, 1, 2, 2, 2, 2, 1],
 [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 1, 2, 1, 0],
 [2, 2, 1, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2],
 [2, 0, 2, 1, 0, 0, 0, 0, 0, 2, 2, 1, 1, 1, 1],
 [2, 0, 2, 0, 1, 1, 0, 0, 0, 0, 1, 2, 2, 1, 2],
 [0, 1, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 0, 0, 2],
 [1, 1, 1, 1, 1, 0, 2, 0, 2, 2, 2, 2, 2, 1, 0],
 [2, 0, 1, 2, 1, 0, 2, 2, 2, 2, 1, 2, 0, 2, 0],
 [0, 1, 0, 0, 1, 1, 2, 1, 0, 1, 1, 0, 2, 2, 2]]